In [1]:
import pandas as pd
from google.cloud import bigquery
import pydata_google_auth

In [2]:
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [3]:
client = bigquery.Client(project= "project-football-388908", credentials=credentials)

In [4]:
# Query Bigquery
project_id = "project-football-388908"
dataset = "Transfer_data"
table = ""
query = f"SELECT * FROM `{project_id}.{dataset}.{table}`"

In [5]:
tables = client.list_tables(f'{project_id}.{dataset}')
table_names = []
for table in tables:
    table_names.append(f'{table.table_id}')
table_names

['PL_transers_in_Big6_2014_2022',
 'PL_transfer_data_final',
 'PL_transfer_data_final_extra',
 'PL_transfer_data_net_big5',
 'PL_transfers',
 'PL_transfers_cleaned',
 'PL_transfers_evleiwestsouth2']

In [6]:
tables = client.list_tables(f'{project_id}.{dataset}')
for table, table_name in zip(tables, table_names):
  query = f"SELECT * FROM `{f'{table.project}.{table.dataset_id}.{table.table_id}' }`"
  vars()[table_name] = client.query(query).to_dataframe()

In [7]:
PL_transfer_data_final["transfer_movement"].value_counts()

out    4660
in     4307
Name: transfer_movement, dtype: int64

In [8]:
PL_transfer_data_final["fee_in"] = PL_transfer_data_final["fee_cleaned_1"]
PL_transfer_data_final.loc[PL_transfer_data_final["transfer_movement"]=="out", "fee_in"] = 0
PL_transfer_data_final

,club_name,player_name,club_involved_name,fee,transfer_movement,league_name,season,fee_cleaned_1,year_1,age_1,transfer_period_1,position_1,fee_in
0,Everton FC,Adam Eaton,Preston,€6Th.,out,Premier League,1998/1999,0.006,1998,19,SUMMER,DEFENCE,0.000
1,Southampton FC,Frankie Bennett,Halesowen,€9Th.,in,Premier League,1992/1993,0.009,1992,24,WINTER,CENTRE-FORWARD,0.009
2,Norwich City,Mark Robins,Man Utd,€1.00m,in,Premier League,1992/1993,1.000,1992,22,SUMMER,CENTRE-FORWARD,1.000
3,Crystal Palace,Chris Armstrong,Millwall,€1.00m,in,Premier League,1992/1993,1.000,1992,21,SUMMER,CENTRE-FORWARD,1.000
4,Manchester United,Mark Robins,Norwich,€1.00m,out,Premier League,1992/1993,1.000,1992,22,SUMMER,CENTRE-FORWARD,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8962,Everton FC,Henry Onyekuru,Galatasaray,Loan fee:€800Th.,out,Premier League,2018/2019,0.800,2018,21,SUMMER,LEFT WINGER,0.000
8963,Norwich City,Joseph Yobo,Fenerbahce,Loan fee:€850Th.,in,Premier League,2013/2014,0.850,2013,33,WINTER,CENTRE-BACK,0.850
8964,West Ham United,Edimilson Fernandes,Fiorentina,Loan fee:€865Th.,out,Premier League,2018/2019,0.865,2018,22,SUMMER,CENTRAL MIDFIELD,0.000
8965,Birmingham City,Christian Benítez,Santos Laguna,Loan fee:€900Th.,in,Premier League,2009/2010,0.900,2009,23,SUMMER,CENTRE-FORWARD,0.900


In [9]:
PL_transfer_data_final["fee_out"] = PL_transfer_data_final["fee_cleaned_1"]
PL_transfer_data_final.loc[PL_transfer_data_final["transfer_movement"]=="in", "fee_out"] = 0

In [10]:
PL_transfer_data_net = PL_transfer_data_final.groupby(by=["club_name","year_1"], as_index=False).agg({"fee_in":"sum", "fee_out": "sum"})

In [11]:
PL_transfer_data_net['Net_spend'] = PL_transfer_data_net['fee_in']-PL_transfer_data_net['fee_out']

In [12]:
PL_transfer_data_net

,club_name,year_1,fee_in,fee_out,Net_spend
0,AFC Bournemouth,2015,55.11,0.650,54.460
1,AFC Bournemouth,2016,40.69,23.690,17.000
2,AFC Bournemouth,2017,34.30,0.000,34.300
3,AFC Bournemouth,2018,89.10,20.200,68.900
4,AFC Bournemouth,2019,54.91,34.625,20.285
...,...,...,...,...,...
621,Wolverhampton Wanderers,2018,112.75,23.300,89.450
622,Wolverhampton Wanderers,2019,116.70,24.100,92.600
623,Wolverhampton Wanderers,2020,87.80,79.200,8.600
624,Wolverhampton Wanderers,2021,37.60,31.800,5.800


In [13]:
PL_transfer_data_net_mid = PL_transfer_data_net[(PL_transfer_data_net["club_name"] == "Leicester City") | (PL_transfer_data_net['club_name'] == 'West Ham United') | (PL_transfer_data_net['club_name'] == 'Southampton FC') | (PL_transfer_data_net['club_name'] == 'Everton FC')]

In [14]:
PL_transfer_data_net_mid

,club_name,year_1,fee_in,fee_out,Net_spend
194,Everton FC,1992,0.703,3.450,-2.747
195,Everton FC,1993,4.055,1.700,2.355
196,Everton FC,1994,12.380,2.250,10.130
197,Everton FC,1995,16.980,0.420,16.560
198,Everton FC,1996,10.750,15.385,-4.635
...,...,...,...,...,...
596,West Ham United,2018,100.900,13.765,87.135
597,West Ham United,2019,119.800,55.480,64.320
598,West Ham United,2020,54.700,45.410,9.290
599,West Ham United,2021,74.500,4.235,70.265


In [15]:
Table = "PL_transfer_data_net_mid"

In [16]:
from pandas.io import gbq

In [17]:
PL_transfer_data_net_mid.to_gbq(destination_table= f"{project_id}.{dataset}.{Table}", project_id=project_id, if_exists="replace", credentials=credentials)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
